In [1]:
#cell 1
# =============================
# 数据准备:原子化拆解
# =============================
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns
from dotenv import load_dotenv
import os

# 加载本地.env文件
load_dotenv()

# 从环境变量读取连接信息
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASS")
HOST = os.getenv("DB_HOST","localhost")
PORT = os.getenv("DB_PORT","5432")
DB = os.getenv("DB_NAME")

# 检查读取是否成功(调试用)
if not PASSWORD:
    print("警告:未找到数据库密码,请检查.env文件")
else:
    print("配置加载成功(具体信息已隐藏)")

engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")

query_item_level = """
WITH 
base_joined AS (
    SELECT
        oi.order_id,
        -- 该product_id对应的商品在订单中是第几个商品(序号)
        oi.order_item_id,
        oi.product_id,
        -- 对应卖家
        oi.seller_id,
        oi.price,
        -- 对应商品名称
        t.product_category_name_english as category,
        
        -- 关联订单级的体验指标(评分、延迟)
        o.review_score,
        o.delay_days,
        
        -- 延迟交付标记
        CASE WHEN o.delay_days > 0 THEN 1 ELSE 0 END AS is_late
    FROM Olist.olist_order_items_dataset oi
    -- 关联OBT
    INNER JOIN analysis.analysis_orders_obt o
        ON oi.order_id = o.order_id
    -- 关联商品表
    LEFT JOIN Olist.olist_products_dataset p
        ON oi.product_id = p.product_id
    -- 关联翻译表
    LEFT JOIN Olist.product_category_name_translation t
        ON p.product_category_name = t.product_category_name
)
SELECT * FROM base_joined;
"""

# 拉取全量原子数据
df_items = pd.read_sql(query_item_level, engine)
print(f"✅ 原子化数据准备完成。行数: {len(df_items)} (比订单数多，因为包含了 Mixed Basket)")
df_items.head()

配置加载成功(具体信息已隐藏)
✅ 原子化数据准备完成。行数: 110189 (比订单数多，因为包含了 Mixed Basket)


,order_id,order_item_id,product_id,seller_id,price,category,review_score,delay_days,is_late
0,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,pet_shop,4,-2.0,0
1,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,furniture_decor,5,-13.0,0
2,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,garden_tools,5,-15.0,0
3,00048cc3ae777c65dbb7d2a0634bc1ea,1,ef92defde845ab8450f9d70c526ef70f,6426d21aca402a131fc0a5d0960a3c90,21.90,housewares,4,-14.0,0
4,00054e8431b9d7675808bcb819fb4a32,1,8d4f2bb7e93e6710a28f34fa83ee7d28,7040e82f899a04d1b434b795a43b4617,19.90,telephony,4,-16.0,0
